# 실전 문제 해결 (과소적합)

In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## 하이퍼 파라미터

In [2]:
EPOCHS = 100

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, use_bias=False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias=False)
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias=False)

        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias=False)
        
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h], axis=-1)
        
        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h], axis=-1)
        
        return self.dense5(x)

## 데이터셋 준비


In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist #28x28 -> 10 class

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습

In [ ]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 23s 11ms/step - loss: 0.5129 - accuracy: 0.8149 - val_loss: 0.4549 - val_accuracy: 0.8316
Epoch 2/100
1875/1875 [==============================] - 40s 21ms/step - loss: 0.3979 - accuracy: 0.8556 - val_loss: 0.4144 - val_accuracy: 0.8535
Epoch 3/100
1875/1875 [==============================] - 42s 22ms/step - loss: 0.3658 - accuracy: 0.8664 - val_loss: 0.3847 - val_accuracy: 0.8616
Epoch 4/100
1875/1875 [==============================] - 41s 22ms/step - loss: 0.3434 - accuracy: 0.8741 - val_loss: 0.4042 - val_accuracy: 0.8542
Epoch 5/100
1875/1875 [==============================] - 39s 21ms/step - loss: 0.3326 - accuracy: 0.8772 - val_loss: 0.4337 - val_accuracy: 0.8490
Epoch 6/100
1875/1875 [==============================] - 40s 21ms/step - loss: 0.3197 - accuracy: 0.8830 - val_loss: 0.3764 - val_accuracy: 0.8640
Epoch 7/100
1875/1875 [==============================] - 41s 22ms/step - loss: 0.3078 - accuracy: 0.8871 - val_loss: 0